In [ ]:
%load_ext autoreload
%autoreload 2

# 🧠 TissueTag Annotation Tutorial: Visium HD H&E Image of Mouse Brain

Welcome!  
In this tutorial, we’ll walk through the process of annotating a **Visium HD H&E image** from a **fresh-frozen mouse brain sample**. This notebook will guide you from raw data download to generating high-quality tissue annotations using both automated and manual approaches.

## 📦 Download the Sample Data

We'll use the publicly available dataset from 10x Genomics.  
Run the following commands in your terminal to download all necessary files and set up the demo directory

<small>

```bash
mkdir ../data/tissue_tag_example_visiumHD
chdir ../data/tissue_tag_example_visiumHD
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.1/Visium_HD_Mouse_Brain_Fresh_Frozen/Visium_HD_Mouse_Brain_Fresh_Frozen_web_summary.html
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.1/Visium_HD_Mouse_Brain_Fresh_Frozen/Visium_HD_Mouse_Brain_Fresh_Frozen_cloupe_008um.cloupe
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.1/Visium_HD_Mouse_Brain_Fresh_Frozen/Visium_HD_Mouse_Brain_Fresh_Frozen_feature_slice.h5
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.1/Visium_HD_Mouse_Brain_Fresh_Frozen/Visium_HD_Mouse_Brain_Fresh_Frozen_metrics_summary.csv
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.1/Visium_HD_Mouse_Brain_Fresh_Frozen/Visium_HD_Mouse_Brain_Fresh_Frozen_molecule_info.h5
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.1/Visium_HD_Mouse_Brain_Fresh_Frozen/Visium_HD_Mouse_Brain_Fresh_Frozen_spatial.tar.gz
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.1.1/Visium_HD_Mouse_Brain_Fresh_Frozen/Visium_HD_Mouse_Brain_Fresh_Frozen_binned_outputs.tar.gz
tar -xvzf Visium_HD_Mouse_Brain_Fresh_Frozen_spatial.tar.gz
tar -xvzf Visium_HD_Mouse_Brain_Fresh_Frozen_binned_outputs.tar.gz
``` 
</small>


## 🧭 Annotation Strategy

We’ll explore several annotation strategies, ranging from automated predictions to manual refinement:

1. **Fully Automatic** – Quickly capture broad tissue features (e.g., tissue boundaries, white and gray matter) using a simple pixel classifier.  
2. **Semi-Automatic** – Gene-guided labeling combined with manual annotation and drawing for finer control.

All annotations will be saved as a `TissueTag` annotation class object in an `.h5` file.


---

Let's get started! 🚀


In [ ]:

# initialisation 
import os
import panel as pn
import bin2cell as b2c
import tissue_tag as tt
import tissue_tag.annotation
import tissue_tag.io
from tissue_tag.io import TissueTagAnnotation
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*"
# host = <port> # set the port to the value in the address bar when operating in farm
host = '8888' # when working locally e.g. desktop

# 🚀 Initiate

We will start by:

- Reading in the image data  
- Defining the resolution we’ll work with  
- Loading the associated `AnnData` object  

> ⚠️ **Note:** `TissueTag` is designed for **tissue-level** annotations. We do **not** recommend using it for annotating very small structures, such as single cells.  
> For best performance and clarity, we recommend working at a resolution of **0.5 pixels per micron (ppm)** — equivalent to **2 microns per pixel**.


In [ ]:
# set paths
# Here we will use the 16um binned data that is available from 10X for the annotation, don't worry, later you can map these annotations to any other resolution (2um, 8um and even a single cell Bin2Cell object - https://github.com/Teichlab/bin2cell )
spaceranger_dir_path = "../data/tissue_tag_example_visiumHD"
mapped_image_path = spaceranger_dir_path + '/Visium_HD_Mouse_Brain_Fresh_Frozen_tissue_image.tif' # the image that was used for spaceranger input
spaceranger_spatial_path = spaceranger_dir_path +'/spatial' # path to the spatial folder for this dataset (this structure was changed a few times) 
bin_resolution = 16 # bin resolution to work with

tt_obj = tissue_tag.io.read_visium_hd(
    spaceranger_dir_path=spaceranger_dir_path, 
    mapped_image_path=mapped_image_path,
    use_resolution='mapped_res', # use the max resolution 
    ppm_out=0.5,
    plot=True,
    bin_resolution=bin_resolution)

# Automatic annotations - Create de-novo annotations from gene expression (or not)

In [ ]:
# read anndata with bin2cell make sure you are using the same binning resolution!!
adata = b2c.read_visium(
    path = spaceranger_dir_path + f'/binned_outputs/square_0{bin_resolution}um/',
    spaceranger_image_path = spaceranger_spatial_path
)

In [ ]:
# Define anatomical structure annotations
# and their associated color codes.
#
# Color families:
#   • Red:         'red', 'darkred', 'firebrick', 'indianred'
#   • Green:       'green', 'darkgreen', 'lime', 'seagreen', 'forestgreen'
#   • Blue:        'blue', 'darkblue', 'royalblue', 'dodgerblue', 'deepskyblue'
#   • Cyan:        'cyan', 'lightcyan', 'darkcyan', 'teal'
#   • Magenta:     'magenta', 'purple', 'darkmagenta', 'orchid', 'violet'
#   • Yellow/Orange:'gold', 'orange', 'darkorange', 'goldenrod'
#   • Brown:       'brown', 'saddlebrown', 'chocolate', 'peru', 'tan'
#   • Gray/Black:  'black', 'gray', 'darkgray', 'dimgray', 'lightgray'
#   • White:       'white'

tt_obj.annotation_map = {
    'unassigned':      'yellow',
    'isocortex':       'green',
    'hippocampus':     'darkgreen',
    'olfactory':       'orange',
    'striatum':        'red',
    'thalamus':        'blue',
    'amygdala':        'lime',
    'choroid_plexus':  'gold',
    'pia':             'deepskyblue',
    'white_matter':    'white',
    'gray_matter':     'teal',
    'dentate_gyrus':   'violet',
    'layer_1':         'tan',

}
# note if you need to add annotation to an exxisting object add them in the end and do not change the order as this corresponds to the pixel values of the label image. e.g. unassigned = 1, isocortex=2 etc

In [ ]:

# Define gene markers per region
# Format: region_name: List of (gene, expression threshold)

gene_markers = {
    'gray_matter': [
        ('Gad1', 2500),
        ('Gad2', 2500),
        ('Slc17a7', 3000)
    ],
    'white_matter': [
        ('Mbp', 1500),
        ('Gfap',500)
    ]
}

# Generate training labels from gene expression
# This maps regions based on marker expression

tissue_tag.annotation.gene_labels_from_adata(
    adata=adata,
    gene_markers=gene_markers,
    tissue_tag_annotation=tt_obj,
    diameter=bin_resolution * 4,  # Labeling diameter
    override_labels=True,         # Replace any existing labels
    normalize=True               # Use raw expression
)


# Visualize the assigned labels
tissue_tag.annotation.median_filter(tt_obj,filter_radius=bin_resolution)
tissue_tag.annotation.plot_labels(tt_obj, alpha=0.5) # i wasn't able to supress putput in annotation.plot_labels



### 🧠 Pixel Classification

In this step, we train a **Random Forest pixel classifier** using the labeled regions provided earlier.  
This model will then be used to predict annotations for all image pixels.

⏱️ **Estimated runtime:** 1–10 minutes, depending on image resolution, sysyem performance and the number of training areas.

The result will be stored in the existing `TissueTag` object.


In [ ]:
%%time
tissue_tag.annotation.pixel_label_classifier(tt_obj)

# 🧬 Gene-Guided Labeling

In this step, we overlay **gene expression–based labels** on top of the previous classification.  
These labels serve as additional guidance for **manual tissue annotation**, helping refine regional identity using known gene markers.

We define a dictionary of **marker genes** and their expression thresholds for key brain regions.  
From this, we generate spatial labels directly from the `AnnData` object and add them to the existing `TissueTag` annotation.

> 🔍 These gene-based labels are not final annotations—they are **hints** to support accurate manual refinement.


In [ ]:
# select gene markers 
gene_markers = {
    'isocortex': [('Pak7',500),('Myl4',500),('Ttc9b',500)],
    'amygdala':[('Acvr2a',300)],
    'olfactory': [('Cdhr1',500)],
    'striatum': [('Adora2a',200),('Gprin3',200)],
    'thalamus': [('Plekhg1',500)],
	'choroid_plexus':[('Tcf21',500)],
    'hippocampus': [('Zbtb20',500)],
}

# generate training data from gene expression
tissue_tag.annotation.gene_labels_from_adata(
    adata = adata,
    gene_markers = gene_markers,    
    tissue_tag_annotation=tt_obj,
    diameter = bin_resolution*2,
    normalize=False,

) # generate gene-marker-labels
tissue_tag.annotation.plot_labels(tt_obj, alpha=0.25)

### ✍️ Interactive Tissue Annotator

Now it's time to manually refine the tissue annotation using the **interactive annotator**, powered by **Panel** and **Datashader**.

This tool gives you full control over the labeling process with an intuitive interface:

1. 🖼️ **Image Toggle**  
   Use the slider to toggle between the raw image and the current annotation labels for visual comparison.

2. 🎨 **Structure Selector**  
   On the left, you'll see each tissue structure represented by its **first letter** in color.  
   Hover over a letter to reveal the **full region name**.

3. 🖋️ **Drawing Annotations**  
   Select a tissue structure and **draw freely** over the region of interest.  
   Then, click the **Update** button to "fill" the sketched shapes into annotations.  
   *(Shapes do not need to be closed manually—this is handled for you.)*

4. 🗑️ **Removing Annotations**  
   - If you haven’t pressed **Update** yet, simply click the drawn line and hit `Backspace`.  
   - If you've already updated, you can click **Revert** to undo the last annotation fill.

---

> ⚙️ **Rendering Option: `use_datashader=True`**  
> At this stage, you can choose whether to render the image using **Datashader**.  
> Setting `use_datashader=True` is **recommended for large images or high-resolution annotations**.  
> While annotation drawing is slightly slower with Datashader, it ensures smooth and responsive image loading.  
> Without Datashader, very large images might fail to load or take a very long time to render.

---

> ⚠️ **Tip:** For the best experience, we **highly recommend using a mouse with a scroll wheel**. Trackpads or touchscreen input may behave inconsistently.

All annotations are automatically assigned to your `TissueTag` annotation object and can be saved later as part of the `.h5` file.


In [ ]:
# use annotator to label tissue regions according to categories indicated above
annotator = tissue_tag.annotation.annotator(tt_obj, use_datashader=True)
pn.io.notebook.show_server(annotator, notebook_url=f'localhost:'+host)

In [ ]:
tissue_tag.annotation.plot_labels(tt_obj, alpha=0.5)

# 💾 Save Annotation Output

After completing the annotation process, we save the results for future use.  
This includes the labeled image, annotation map, and any interactive edits made during the session.

Annotations will be saved as an `.h5` file inside the `tt_annotations` folder, within the binned output directory:

- The output path is based on the current bin resolution (e.g., `square_16um` for 16μm bins).
- If the target directory does not exist, it will be created automatically.

This allows seamless integration with downstream analysis or loading annotations in a future session using `TissueTag`.


In [ ]:
path = spaceranger_dir_path + f'/binned_outputs/square_0{bin_resolution}um/'
isExist = os.path.exists(path+'tt_annotations')
if not(isExist):
    os.mkdir(path+'/tt_annotations/')
    
tt_obj.save_annotation(file_path=path+'/tt_annotations/annotations_v1.h5')

In [ ]:
# we recommend loading the object to make sure it's all saved properly 
tt_obj = tt.load_annotation(file_path=path+'/tt_annotations/annotations_v1.h5')
im = tissue_tag.annotation.plot_labels(tt_obj, alpha=0.5)

# 📐 Calculate Spatial Distances to Annotations

In this step, we generate a **regular spatial grid** across the annotated image and compute the **mean distance from each grid point to nearby annotated tissue structures**.

This allows us to:

- Consistently quantify spatial proximity to anatomical regions
- Create interpretable spatial axes (e.g. cortical gradients, medial-lateral transitions)

#### 🧭 How it works:

1. **Grid Generation**  
   A tight hexagonal grid is generated across the tissue using `grid_unit_size` spacing (in microns).  
   Each grid point is assigned a label based on the median annotation value "under" each grid spot.

2. **Neighborhood Distance Calculation**  
   Using a fast KD-tree implementation, the grid is queried against each annotated region.  
   For each annotated structure, we calculate the **mean distance** to its `k` nearest neighbors (`nhood_size`), for every grid point.

---

> ⚙️ **Parameters Used**  
> - `grid_unit_size = 15`: spacing between grid points, in microns  
> - `nhood_size = 10`: number of nearest neighbors used for computing distances  

---

The output is stored in `tt_obj.grid`, containing:

- Coordinates (`x`, `y`)
- Region labels and numeric codes
- Per-category distance columns (e.g. `L2_dist_annotation_hippocampus`)

You can use this grid for further downstream tasks such as:

- Visualizing spatial trends
- Transferring annotations to data (`map_annotations_to_target`) e.g. 16/8/2um/Bin2Cell object from the same visium HD dataset
- Creating spatial axes (with `calculate_axis_2p/3p`)
- Binning space into anatomical domains (`bin_axis`)



In [ ]:
# tt_obj = tt.load_annotation(file_path=path+'/tt_annotations/annotations.h5')
grid_unit_size = 15
nhood_size = 10

########## this part should be merged into a future function tt_obj.grid = calculate_distances_to_nhood(tt_obj,grid_unit_size,nhood_size)
tt_obj.grid = tt.generate_grid_from_annotation(tt_obj, spot_to_spot = grid_unit_size)
print('calculating distances')
tt.calculate_distance_to_annotations(
    tt_obj.grid,
    knn=nhood_size
    ) # calculate minimum mean distance of each spot to clusters
tt_obj.grid['annotation'].value_counts()

##########


### 🔄 Map Grid Annotations to Visium HD 16μm Spots

Now that we’ve generated the annotated spatial grid, we map these annotations back to the original **Visium HD 16μm object** to enable downstream integration with gene expression data.

#### 🧭 Step Overview:

1. **Extract Spot Coordinates**  
   We directly convert `adata.obsm['spatial']` into a DataFrame with `"x"` and `"y"` columns, using the spot barcodes from `adata.obs_names` as the index.

2. **Calculate Pixels-Per-Micron (ppm)**  
   The spatial scaling is retrieved from `adata.uns['spatial']`, and the pixels-per-micron (`ppm_target`) is computed as the inverse of `"microns_per_pixel"`.

3. **Map Annotations**  
   We use `tt.map_annotations_to_target()` to assign annotations from the grid to Visium spots using a **KD-tree nearest-neighbor search**.  
   The `max_distance` parameter ensures that only nearby annotations (within 2× spot size) are transferred.

4. **Alignment Preview**  
   A scatter plot is automatically generated to visually compare the source grid and target Visium spot positions, ensuring spatial alignment.

5. **Map to AnnData obs**  
   matching back indices and transfer annotations to object 

---

> 🧪 This step is essential to bridge spatial gene expression data with your refined anatomical annotation grid.

> 📌 `ppm_source` is set to 1 (since the grid is already in pixel space), while_



In [ ]:
# Display the resulting DataFrame
import pandas as pd
tt_obj.grid = tt_obj.grid[tt_obj.grid['annotation']!='unassigned']

ppm_target = 1/adata.uns['spatial']['Visium_HD_Mouse_Brain_Fresh_Frozen']['scalefactors']['microns_per_pixel']
vis_df = tt.map_annotations_to_target(    
    df_target=pd.DataFrame(adata.obsm['spatial'], index=adata.obs_names, columns=["x", "y"]),
    df_source=tt_obj.grid,
    ppm_source=1, # this is always 1
    ppm_target=ppm_target, # do one over for ppm,
    plot=True,
    max_distance=int(grid_unit_size*2/ppm_target)
)      

# Ensure both DataFrames have the same index type
vis_df = vis_df.loc[adata.obs.index]  # Align indices to avoid mismatches

# Add columns from the fourth onward to adata_vis.obs
adata.obs = pd.concat([adata.obs, vis_df.iloc[:, 2:]], axis=1)
adata.obs = adata.obs.loc[:, ~adata.obs.columns.duplicated()]
# Display the updated adata_vis.obs
adata.obs

# 🧭 Compute Spatial Axes Between Anatomical Landmarks

To better capture spatial organization within complex brain regions, we calculate **continuous spatial axes** that represent each spot’s position **relative to two anatomical landmarks**.

#### 📐 Axis Calculation

Using the `tt.calculate_axis_2p()` function, we generate **directional, normalized spatial gradients** between selected anatomical regions:

- `pia_to_wm`: Measures depth along the **cortical column**, from the pia surface to underlying white matter.
- `olf_to_wm`: Captures positioning along the **olfactory–white matter axis**, useful for modeling transitions in **striatal** regions.
- `iso_to_wm`: Represents spatial location **within the isocortex**, from the cortical-corpus callosum interface as a 0 point.
- `hip_to_den`: Describes a spatial axis within the **hippocampus**, spanning from hippocampal tissue toward the **dentate gyrus**.  
  *(This one is exploratory and not grounded in known biological gradients—just for fun!) 😉

Each computed axis is added as a new column in `adata.obs`, enabling further stratification, visualization, and spatial modeling.


#### 🎯 Contextual Filtering

To improve specificity and interpretability, we apply **region-aware masking** to restrict each spatial axis to its relevant anatomical context:

- `pia_to_wm` is retained only in **isocortex** or **layer_1**
- Spots located far from both **pia** and **white matter** (> ~1000 microns\*) are excluded
- `olf_to_wm` is restricted to the **striatum**
- `iso_to_wm` is limited to **isocortex** and **white matter**, and further filtered to spots within ~300 microns\* of either structure
- `hip_to_den` is retained only in **hippocampus** or **dentate gyrus**

\* _Note:_ These distances are calculated from the **mean distance to a set of nearby points (neighborhood)**, not from a single closest spot.  
If you need distances that reflect the **exact distance to the single nearest spot**, set `nhood_size=1` when computing distances.  
However, keep in mind that setting `nhood_size=1` may limit your ability to calculate smooth spatial axes **within structures**—as done in `iso_to_wm` and `hip_to_den`.




In [ ]:

tt.calculate_axis_2p(adata.obs, structure = ['pia','white_matter'], output_col='pia_to_wm')
tt.calculate_axis_2p(adata.obs, structure = ['olfactory','white_matter'], output_col='olf_to_wm')
tt.calculate_axis_2p(adata.obs, structure = ['isocortex','white_matter'], output_col='iso_to_wm')
tt.calculate_axis_2p(adata.obs, structure = ['hippocampus','dentate_gyrus'], output_col='hip_to_den')


# here we will do 
adata.obs.loc[~adata.obs['annotation'].isin(['isocortex','layer_1']),'pia_to_wm'] = None

mask = (
    (adata.obs['L2_dist_annotation_white_matter'] > 1000) &
    (adata.obs['L2_dist_annotation_pia'] > 1000)
)
adata.obs.loc[mask, 'pia_to_wm'] = None
adata.obs.loc[~adata.obs['annotation'].isin(['striatum']),'olf_to_wm'] = None
adata.obs.loc[~adata.obs['annotation'].isin(['isocortex','white_matter']),'iso_to_wm'] = None
adata.obs.loc[adata.obs['L2_dist_annotation_white_matter']>300,'iso_to_wm'] = None
adata.obs.loc[adata.obs['L2_dist_annotation_isocortex']>300,'iso_to_wm'] = None

adata.obs.loc[~adata.obs['annotation'].isin(['hippocampus','dentate_gyrus']),'hip_to_den'] = None


In [ ]:
# plot the newly annotated visium AnnData
import scanpy as sc
sc.set_figure_params(figsize=[5,5],dpi=100)
sc.pl.spatial(adata,color=['pia_to_wm','olf_to_wm','hip_to_den','iso_to_wm'],cmap='gist_rainbow',ncols=2)

In [ ]:

# Get all obs columns that contain "L2_dist"
l2_dist_cols = [col for col in adata.obs.columns if 'L2_dist' in col]

# Plot them using scanpy
sc.set_figure_params(figsize=[5, 5], dpi=100)
sc.pl.spatial(adata, color=l2_dist_cols, cmap='jet', ncols=5)


In [ ]:
sc.pl.spatial(adata,color=['annotation'],ncols=3) # andrian march the colors!

In [ ]:
# downstream analysis 
sc.pp.filter_cells(adata,min_counts=100)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [ ]:
adata.var_names_make_unique()

In [ ]:
sc.pp.highly_variable_genes(adata,n_top_genes=5000)
adata = adata[:, adata.var["highly_variable"]].copy()
sc.pp.scale(adata, max_value=10)
sc.pp.pca(adata, use_highly_variable=True)
sc.pp.neighbors(adata)
sc.tl.umap(adata)



In [ ]:
sc.pl.umap(adata,color=['pia_to_wm','olf_to_wm','hip_to_den','iso_to_wm'],cmap='gist_rainbow',ncols=2,s=2)

In [ ]:
sc.pl.umap(adata,color=['annotation'],ncols=2,s=2)